<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf


    Found existing installation: protobuf 6.31.1
    Uninstalling protobuf-6.31.1:
      Successfully uninstalled protobuf-6.31.1


  Attempting uninstall: yfinance


    Found existing installation: yfinance 0.2.65
    Uninstalling yfinance-0.2.65:
      Successfully uninstalled yfinance-0.2.65
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-07-09 12:42:50
-------------------
qualified stocks: 96
with latest results: 31
still star stocks: 20
-------------------
Initial Investment:  1.17 C
CY Investment:  1.44 C
Reserve:  9.00 K
Current:  1.37 C
-------------------
Today PnL: 44.22 K (0.32%)
Current PnL: -12.81 L (-8.92%)
CY Booked + Current PnL: -5.85 L (-4.07%)
-------------------
Total profit:  5.08 L
Total loss:  -17.89 L
-------------------
Total Booked + Current PnL: 21.13 L (18.13%)
Total Booked PnL: 33.94 L (29.12%)
Curr Year Booked PnL: 6.96 L (5.07%)
Prev Year Booked PnL: 26.98 L (23.15%)
Est FTT:  2.09 C
Est FTT PnL: 71.30 L (51.93%)
Deployed:  1.17 C
Current:  1.37 C
CAGR/XIRR %: 11.05%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'RSP', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
92,VIPIND,488.80,489.00,-218.02,H-SC,98.96,85147.0,-10169.0,10209.0,3.64,-10.67,11.99,0.04,157.0,-1.00,0.62,67.94,OX40N,NTT,MISC
4,APOLLOHOSP,6904.43,8285.00,-16.74,H-LC,89.58,113475.0,9909.0,10803.0,-0.67,9.57,9.52,20.00,32.0,0.92,0.83,24.99,X40N,BTT,HEALTHCARE
71,SBIN,760.30,863.00,-11.51,M-LC,32.29,214782.0,13302.0,13918.0,-0.26,6.60,6.48,13.51,77.0,0.96,1.57,20.03,XY25,NTT,BANKS
39,HINDALCO,651.95,761.55,-22.38,H-LC,69.79,107640.0,3328.0,14208.0,-1.87,3.19,13.20,16.81,27.0,0.23,0.78,19.75,X5K,ATH,METALS
87,UJJIVANSFB,52.77,53.00,45.86,M-SC,22.92,126981.0,-15498.0,16114.0,-0.21,-10.88,12.69,0.44,247.0,-0.96,0.93,49.87,OX40N,NTT,BANKS


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
12,BANKINDIA,116.91,190.00,-32.52,M-MC,7.29,178577.0,-1231.0,113646.0,-0.73,-0.68,63.64,62.52,173.0,-0.01,1.30,33.17,XR,NTT,BANKS
38,HEROMOTOCO,4311.81,6039.03,-5.71,H-MC,38.54,151736.0,823.0,59632.0,0.81,0.55,39.30,40.06,103.0,0.01,1.11,24.11,AR,ATH,AUTO


In [11]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
59,MEDANTA,1064.65,1486.00,18.07,X-SC,90.62,251672.0,46195.0,35133.0,7.05,22.48,13.96,39.58,65.0,1.31,1.83,34.86,XY24,NTT,HEALTHCARE
92,VIPIND,488.80,489.00,-218.02,H-SC,98.96,85147.0,-10169.0,10209.0,3.64,-10.67,11.99,0.04,157.0,-1.00,0.62,67.94,OX40N,NTT,MISC
32,ENRIN,1377.95,3143.60,NaN,NaN,95.83,78590.0,44141.0,0.0,3.17,128.14,0.00,128.14,274.0,inf,0.57,20.18,AR,ATH,ELECTRICAL
54,KPIGREEN,497.21,731.84,28.70,H-SC,55.21,131494.0,6197.0,52926.0,2.88,4.95,40.25,47.19,143.0,0.12,0.96,64.51,MH,ATH,POWER
94,WHIRLPOOL,1167.49,2270.00,-38.94,M-SC,52.08,201449.0,29828.0,132251.0,2.81,17.38,65.65,94.43,201.0,0.23,1.47,48.68,XR,NTT,DURABLES


In [12]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
88,UNIONBANK,123.87,163.00,-6.01,M-LC,5.21,164171.0,23331.0,21162.0,-3.90,16.57,12.89,31.59,88.0,1.10,1.20,42.61,XY24,NTT,BANKS
28,DIGITIDE,188.38,237.72,NaN,NaN,21.88,49325.0,8070.0,2738.0,-3.31,19.56,5.55,26.19,272.0,2.95,0.36,16.13,XY24,ATH,IT
41,HINDZINC,514.80,730.22,12.55,H-LC,1.04,84540.0,-18420.0,61503.0,-3.09,-17.89,72.75,41.85,24.0,-0.30,0.62,11.27,X5K,ATH,METALS
76,SONACOMS,578.05,1006.00,-38.81,M-SC,2.08,78890.0,-22269.0,97161.0,-2.28,-22.01,123.16,74.03,197.0,-0.23,0.57,11.24,AR,BTT,AUTO
0,5PAISA,564.75,565.00,150.00,H-SC,18.23,134717.0,-52215.0,52297.0,-2.22,-27.93,38.82,0.04,149.0,-1.00,0.98,28.88,OX40N,NTT,FINANCE


In [13]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
92,VIPIND,488.80,489.0,-218.02,H-SC,98.96,85147.0,-10169.0,10209.0,3.64,-10.67,11.99,0.04,157.0,-1.00,0.62,67.94,OX40N,NTT,MISC
87,UJJIVANSFB,52.77,53.0,45.86,M-SC,22.92,126981.0,-15498.0,16114.0,-0.21,-10.88,12.69,0.44,247.0,-0.96,0.93,49.87,OX40N,NTT,BANKS
79,SYMPHONY,1306.42,1306.0,-29.37,M-SC,26.04,150676.0,-20465.0,20417.0,-0.33,-11.96,13.55,-0.03,191.0,-1.00,1.10,8.29,OX40N,NTT,DURABLES
46,INDIGOPNTS,1407.73,1408.0,127.12,H-SC,92.71,148775.0,-25784.0,25812.0,-0.04,-14.77,17.35,0.02,133.0,-1.00,1.08,29.01,OX40N,NTT,PAINTS
53,KANSAINER,299.63,340.0,-67.65,H-SC,54.17,225675.0,-43992.0,80318.0,0.32,-16.31,35.59,13.47,137.0,-0.55,1.64,14.14,XY24,NTT,PAINTS


In [14]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
83,TCS,3794.03,4998.00,-19.02,X-LC,31.25,263484.0,-32450.0,126367.0,-0.83,-10.97,47.96,31.73,1.0,-0.26,1.92,5.46,X40,BTT,IT
48,INFY,1461.46,2275.00,-9.19,X-LC,71.88,291788.0,30187.0,115431.0,-0.52,11.54,39.56,55.67,2.0,0.26,2.13,18.35,X40,BTT,IT
40,HINDUNILVR,2413.81,2723.00,-13.14,X-LC,64.58,271846.0,1499.0,33138.0,1.44,0.55,12.19,12.81,5.0,0.05,1.98,13.69,XY25,NTT,FMCG
17,BRITANNIA,4983.33,6446.05,17.04,X-LC,63.54,268502.0,39269.0,28005.0,-0.03,17.13,10.43,29.35,10.0,1.40,1.96,27.58,XY25,ATH,FMCG
74,SIEMENS,4406.90,7969.85,-3.94,X-LC,18.23,81250.0,-28922.0,117999.0,-1.42,-26.25,145.23,80.85,11.0,-0.25,0.59,19.67,AR,ATH,ELECTRICAL


In [15]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
32,ENRIN,1377.95,3143.6,NaN,NaN,95.83,78590.0,44141.0,0.0,3.17,128.14,0.00,128.14,274.0,inf,0.57,20.18,AR,ATH,ELECTRICAL
59,MEDANTA,1064.65,1486.0,18.07,X-SC,90.62,251672.0,46195.0,35133.0,7.05,22.48,13.96,39.58,65.0,1.31,1.83,34.86,XY24,NTT,HEALTHCARE


In [16]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
68,SAIL,130.64,228.00,93.57,M-MC,33.33,232556.0,7594.0,160068.0,1.18,3.38,68.83,74.53,189.0,0.05,1.69,35.02,XY24,BTT,STEEL
54,KPIGREEN,497.21,731.84,28.70,H-SC,55.21,131494.0,6197.0,52926.0,2.88,4.95,40.25,47.19,143.0,0.12,0.96,64.51,MH,ATH,POWER
20,CAMS,3643.00,5250.99,4.09,H-SC,27.08,116964.0,14960.0,30060.0,1.35,14.67,25.70,44.14,126.0,0.50,0.85,33.29,X40N,ATH,MISC
15,BLUESTARCO,1646.70,2337.55,-3.59,H-SC,96.88,180770.0,16100.0,52984.0,-0.13,9.78,29.31,41.95,124.0,0.30,1.32,17.97,X40N,ATH,AC
95,WIPRO,243.46,420.00,-3.77,M-LC,83.33,164703.0,13758.0,95692.0,-1.48,9.11,58.10,72.51,71.0,0.14,1.20,13.45,XR,NTT,IT


In [17]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
32,ENRIN,1377.95,3143.60,NaN,NaN,95.83,78590.0,44141.0,0.0,3.17,128.14,0.00,128.14,274.0,inf,0.57,20.18,AR,ATH,ELECTRICAL
94,WHIRLPOOL,1167.49,2270.00,-38.94,M-SC,52.08,201449.0,29828.0,132251.0,2.81,17.38,65.65,94.43,201.0,0.23,1.47,48.68,XR,NTT,DURABLES
95,WIPRO,243.46,420.00,-3.77,M-LC,83.33,164703.0,13758.0,95692.0,-1.48,9.11,58.10,72.51,71.0,0.14,1.20,13.45,XR,NTT,IT
54,KPIGREEN,497.21,731.84,28.70,H-SC,55.21,131494.0,6197.0,52926.0,2.88,4.95,40.25,47.19,143.0,0.12,0.96,64.51,MH,ATH,POWER
45,INDIAMART,2327.09,4850.92,-47.33,H-SC,76.04,136941.0,13605.0,120152.0,1.18,11.03,87.74,108.45,123.0,0.11,1.00,35.85,AR,ATH,MISC


In [18]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
16,BLUSPRING,226.45,86.0,NaN,NaN,58.33,18834.0,-30759.0,0.0,1.51,-62.02,0.00,-62.02,270.0,-inf,0.14,26.30,XY24,ATH,MISC
24,COFFEEDAY,59.14,80.0,-55.73,L-SC,75.00,66048.0,-47501.0,87553.0,2.53,-41.83,132.56,35.27,265.0,-0.54,0.48,61.65,XR,NTT,HOTELS
32,ENRIN,1377.95,3143.6,NaN,NaN,95.83,78590.0,44141.0,0.0,3.17,128.14,0.00,128.14,274.0,inf,0.57,20.18,AR,ATH,ELECTRICAL
6,ASIANTILES,75.41,137.0,7166.67,M-SC,78.12,79728.0,-14082.0,90699.0,0.45,-15.01,113.76,81.67,245.0,-0.16,0.58,54.40,XR,NTT,CERAMICS
92,VIPIND,488.80,489.0,-218.02,H-SC,98.96,85147.0,-10169.0,10209.0,3.64,-10.67,11.99,0.04,157.0,-1.00,0.62,67.94,OX40N,NTT,MISC


In [19]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
52,JPPOWER,18.73,26.20,-15.28,L-SC,100.00,170014.0,28172.0,28392.0,0.27,19.86,16.70,39.88,262.0,0.99,1.24,72.29,XY24,NTT,POWER
92,VIPIND,488.80,489.00,-218.02,H-SC,98.96,85147.0,-10169.0,10209.0,3.64,-10.67,11.99,0.04,157.0,-1.00,0.62,67.94,OX40N,NTT,MISC
3,ALKYLAMINE,2347.98,4546.37,8.27,X-SC,97.92,100745.0,-218.0,94751.0,-0.55,-0.22,94.05,93.63,67.0,-0.00,0.73,50.06,SR,ATH,CHEMICALS
15,BLUESTARCO,1646.70,2337.55,-3.59,H-SC,96.88,180770.0,16100.0,52984.0,-0.13,9.78,29.31,41.95,124.0,0.30,1.32,17.97,X40N,ATH,AC
32,ENRIN,1377.95,3143.60,NaN,NaN,95.83,78590.0,44141.0,0.0,3.17,128.14,0.00,128.14,274.0,inf,0.57,20.18,AR,ATH,ELECTRICAL


In [20]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.00
1,25,42.03
2,50,70.68


In [21]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    43.53
LC    33.81
MC    21.56
Name: CurrAlloc%, dtype: float64

In [22]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     31.04
X40      12.94
XY25     12.06
X40N     11.95
XR        9.99
AR        8.16
OX40N     8.14
MH        1.68
X200      1.40
X5K       1.40
SR        1.21
Name: CurrAlloc%, dtype: float64

In [23]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    20.50
M-SC    15.85
H-LC    14.09
X-LC    12.88
H-MC     9.59
X-MC     7.57
M-LC     5.80
X-SC     4.55
M-MC     4.05
L-SC     2.63
L-LC     1.04
L-MC     0.35
Name: CurrAlloc%, dtype: float64

In [24]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,15.26,0.08,33.33
IT,12.02,-10.14,64.28
FINANCE,8.51,-11.68,52.20
BANKS,7.64,-8.95,53.40
MISC,5.88,-19.21,74.71
PAINTS,5.69,-11.76,33.78
ELECTRICAL,5.45,1.56,44.98
HEALTHCARE,4.77,1.62,27.12
AUTO,4.28,-15.05,65.64


In [25]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2630498.0,26
XR,1085877.0,13
AR,1026691.0,10
X40,680990.0,10
X40N,523659.0,11
XY25,464028.0,8
OX40N,311528.0,11
SR,177622.0,2
MH,84973.0,2


In [26]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,1969630.0,20
M-SC,1766372.0,21
H-LC,609400.0,12
X-LC,543663.0,8
H-MC,518316.0,8
M-MC,413490.0,3
X-MC,411456.0,6
X-SC,291997.0,5
M-LC,266167.0,4


In [27]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                 
H-SC       XY24      885852.0      6
M-SC       XY24      805941.0      7
H-SC       AR        523812.0      3
M-SC       XR        333696.0      4
M-MC       XY24      299844.0      2
H-MC       XY24      295253.0      3
X-LC       X40       277248.0      3
H-SC       XR        216362.0      3
           X40N      202360.0      4
H-LC       X40N      193158.0      4
M-SC       AR        187884.0      2
           OX40N     166809.0      5
M-LC       XY24      156557.0      2
X-MC       X40       155089.0      2
H-LC       X40       154708.0      3
L-SC       XR        149594.0      2
X-SC       XR        122705.0      1
M-SC       XY25      119919.0      1
X-LC       AR        117999.0      1
M-MC       XR        113646.0      1
X-MC       XY25      102714.0      1
X-LC       XY25       99057.0      3
M-LC       XR         95692.0      1
X-SC       SR         94751.0      1
H-SC       OX40N      88318.0      3
X-MC       XY24       82921.0      1
M-SC       SR         82871.0      1
H-MC       XY25       81648.0      1
H-LC       AR         79283.0      1
           X5K        75711.0      2
M-SC       X40        69252.0      1
H-LC       X200       68673.0      1
H-MC       AR         59632.0      1
L-SC       AR         58081.0      1
L-MC       XR         54182.0      1
X-MC       X40N       53739.0      1
H-SC       MH         52926.0      1
X-LC       X40N       49359.0      1
L-LC       XY25       46772.0      1
X-SC       OX40N      39408.0      2
H-LC       XY24       37867.0      1
X-SC       XY24       35133.0      1
H-MC       MH         32047.0      1
L-SC       XY24       28392.0      1
H-MC       X40N       25043.0      1
           X40        24693.0      1
X-MC       OX40N      16993.0      1
M-LC       XY25       13918.0      1

In [28]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 26.0 seconds
